# From SQL to pandas challenge 7

In [13]:
# import libraries
import pandas as pd

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titleauthor": "1F1JOiYXStWacOBca6coNVfyVtoST7ZgD",
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "stores": "1f-GCgip7O93CpbAkYvOsc21eKnSOSHsQ", 
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2", 
    "roysched": "1zPRZPoFPEMKyrNR5VSENeYFHGCBZmxbs", 
    "publishers": "1s9E8_AVOziTrowb3wyh2jg3PV763VOyq",
    "pub_info": "1OEgogcGKy--EpuVj0kqq7lyBZNGW6YSv", 
    "jobs": "1V1Za8hUdXD-vJOyRdX4aQV5wanIff2eM", 
    "employee": "1h9mUjsVqpP74b1w0x7KOw37n_n9Ulkt5", 
    "discounts": "111dvSxMcCsTgOuV1wDSKFJxO1Xcxd9VS", 
    "authors": "1fEF89Nhe61EebAljKlwFwfEuokK0o6aJ"
}

# Read data from Google Drive
authors = pd.read_csv(gd_path(files_id["authors"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
employee = pd.read_csv(gd_path(files_id["employee"]), sep=";")
publishers = pd.read_csv(gd_path(files_id["publishers"]), sep=";")
stores = pd.read_csv(gd_path(files_id["stores"]), sep=";")

## 1. Find the total amount of authors for each state

In [2]:
# Therese

authors.groupby("state").agg("count")[["au_id"]]

,au_id
state,
CA,15
IN,1
KS,1
MD,1
MI,1
OR,1
TN,1
UT,2


In [3]:
# Subin

authors.groupby('state').au_id.count()

state
CA    15
IN     1
KS     1
MD     1
MI     1
OR     1
TN     1
UT     2
Name: au_id, dtype: int64

In [5]:
# Tanya

authors.state.value_counts()

CA    15
UT     2
KS     1
TN     1
OR     1
MI     1
IN     1
MD     1
Name: state, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT COUNT(au_id) , state
FROM authors
group by state;
```

## 2. Find the total amount of authors by each state and order them in descending order

In [4]:
authors_state=authors.groupby("state").agg({"au_id": 'count'})
authors_state.au_id.sort_values(ascending=False)

state
CA    15
UT     2
IN     1
KS     1
MD     1
MI     1
OR     1
TN     1
Name: au_id, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT COUNT(au_id) Total,state
FROM authors
GROUP BY state
ORDER BY (total) DESC;
```

## 3. What's the price of the most expensive title from each publisher?

In [8]:
pd.merge(titles,publishers,on='pub_id').groupby('pub_name')['price'].max('price')

pub_name
Algodata Infosystems    22.95
Binnet & Hardley        21.59
New Moon Books          19.99
Name: price, dtype: float64

In [9]:
titles.groupby("pub_id").agg({"price": 'max'})

,price
pub_id,
736,19.99
877,21.59
1389,22.95


### Hint:

In SQL the syntax is:

```sql
SELECT MAX(price), pub_id
FROM titles
GROUP BY pub_id;
```

## 4. Find out the top 3 stores with the most sales

In [11]:
# Bharati - most sales by quantity

sales.groupby("stor_id").agg({"qty":"sum"}).sort_values(by="qty", ascending=False).head(3)

,qty
stor_id,
7131,130
7066,125
7067,90


In [14]:
# Andreas - most sales by money

sales_title=sales.merge(titles[['title_id','title','price']], on='title_id',how='outer')
sales_title['total']=sales_title['qty']*sales_title['price']
sales_title.merge(stores[['stor_id','stor_name']],on='stor_id',how='outer').groupby('stor_name')['total'].sum().sort_values(ascending=False).rename('Total Sales').head(3)

stor_name
Barnum's                                1821.25
News & Brews                            1486.30
Doc-U-Mat: Quality Laundry and Books    1400.15
Name: Total Sales, dtype: float64

### Hint:

In SQL the syntax is:

```sql
SELECT stor_id, SUM(qty)
FROM sales
GROUP BY stor_id
ORDER BY SUM(qty) DESC
LIMIT 3;
```

## 5. Find the average job level for each job_id from the employees table.
    Order the jobs in ascending order by its average job level.

In [15]:
employee.groupby("job_id").agg({"job_lvl":"mean"})

,job_lvl
job_id,
2,215.000000
3,200.000000
4,227.000000
5,187.285714
6,176.000000
7,142.500000
8,125.250000
9,123.250000
10,101.750000


### Hint:

In SQL the syntax is:

```sql
SELECT AVG(job_lvl), job_id
FROM employee
GROUP BY job_id
ORDER BY AVG(job_lvl);
```

## 6. For each type (business, psychology…), find out how many books each publisher has

In [16]:
# Ana

titles.groupby(['pub_id', 'type'])['title_id'].count()

pub_id  type        
736     business        1
        psychology      4
877     UNDECIDED       1
        mod_cook        2
        psychology      1
        trad_cook       3
1389    business        3
        popular_comp    3
Name: title_id, dtype: int64

In [18]:
# Andreas

titles.groupby(['pub_id', 'type'])['title_id'].size()

pub_id  type        
736     business        1
        psychology      4
877     UNDECIDED       1
        mod_cook        2
        psychology      1
        trad_cook       3
1389    business        3
        popular_comp    3
Name: title_id, dtype: int64

### Hint:

In SQL the syntax is:

```sql
SELECT pub_id, type, COUNT(title_id)
FROM titles
GROUP BY pub_id , type;
```

## 7. Add the average price of each publisher - book type combination from your previous query

In [19]:
# Tanya

titles.groupby(["type","pub_id"]).agg({"title":"count","price":"mean"}).sort_values(by="price", ascending=False)

,,title,price
type,pub_id,,
psychology,877,1,21.590000
business,1389,3,17.310000
trad_cook,877,3,15.963333
popular_comp,1389,3,14.316667
mod_cook,877,2,11.490000
psychology,736,4,11.482500
business,736,1,2.990000
UNDECIDED,877,1,0.000000


### Hint:

In SQL the syntax is:

```sql
SELECT  AVG(price), COUNT(*), pub_id,type
FROM titles
GROUP BY pub_id, type;
```

# BONUS

This is part of the section Advanced Pandas.

## 8. From your previous query, keep only the combinations of publisher - book type with an average price higher than 12

In [20]:
new_df = titles.groupby(["pub_id", "type"]).agg({"price":"mean"})
new_df.loc[new_df["price"] > 12]

price
pub_id type                   
877    psychology    21.590000
       trad_cook     15.963333
1389   business      17.310000
       popular_comp  14.316667

### Hint:

In SQL the syntax is:

```sql
SELECT pub_id,type, COUNT(*)
FROM titles
GROUP BY pub_id, type
HAVING AVG(price) > 12;
```

## 9. Order the results of your previous query by these two criteria:
1. Count of books, descendingly
2. Average price, descendingly

In [21]:
9.1

avg_price_count = titles.groupby(['pub_id','type']).agg(count=('pub_id', 'size'), avg_price=('price', 'mean'))
avg_price_count[(avg_price_count['avg_price'] > 12)].sort_values(by=['count'], ascending=False )

count  avg_price
pub_id type                          
877    trad_cook         3  15.963333
1389   business          3  17.310000
       popular_comp      3  14.316667
877    psychology        1  21.590000

In [22]:
9.2

avg_price_count = titles.groupby(['pub_id','type']).agg(count=('pub_id', 'size'), avg_price=('price', 'mean'))
avg_price_count[(avg_price_count['avg_price'] > 12)].sort_values(by=['avg_price'], ascending=False )

,,count,avg_price
pub_id,type,,
877,psychology,1,21.590000
1389,business,3,17.310000
877,trad_cook,3,15.963333
1389,popular_comp,3,14.316667


### Hint:

In SQL the syntax is:

```sql
SELECT  pub_id, type, COUNT(*), AVG(price)
FROM titles
GROUP BY pub_id, type
HAVING AVG(price) > 12
ORDER BY COUNT(*) DESC, AVG(price) DESC;
```

## 10. Some authors have a contract, while others don't - it's indicated in the "contract" column of the authors table.
     
    Select all the states and cities where there are 2 or more contracts overall

In [23]:
contracts2=authors.groupby(['state', 'city']).agg({"contract": 'sum'})
contracts2[contracts2['contract']>=2]

contract
state city                    
CA    Berkeley               2
      Oakland                4
      Palo Alto              2
UT    Salt Lake City         2

### Hint:

In SQL the syntax is:

```sql
SELECT SUM(contract),city,state
FROM authors
GROUP BY state,city
HAVING SUM(contract)>1;
```